In [277]:
import pandas as pd
import numpy as np
from glob import glob
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re
import nltk
pd.set_option('display.max_colwidth', -1)
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from datetime import datetime, timedelta
from nltk import classify, NaiveBayesClassifier
import random



companies_mapping = {'AZN_stocks.csv':"AstraZeneca", 'RHHBY_stocks.csv':"Roche", 'PFE_stocks.csv':"Pfizer", 
                     'NVS_stocks.csv':"Novartis",'BAYRY_stocks.csv':"BayerPharma", 'MRK_stocks.csv':"Merck", 
                     'GSK_stocks.csv':"GSK", 'SNY_stocks.csv':"Sanofi"}

import plotly.express as px

In [144]:
twitter = glob('input/tweets/*.csv')
df_tweets = pd.concat(pd.read_csv(file).assign(filename = file) for file in twitter)

stock = glob('input/stock/*.csv')
df_stock = pd.concat(pd.read_csv(file).assign(filename = file) for file in stock)

df_stock.filename = df_stock.filename.str.split(pat ="\\", expand = True)[1]
df_stock['company'] = df_stock.filename.map(companies_mapping)

df_covid = pd.read_csv('input/covid_data.csv')
df_tweets.reset_index(inplace = True)




del df_tweets['filename']
del df_tweets['index']

del df_stock['filename']

In [145]:
def return_hashes(row):

    return list(set([re.sub('[^\w\s]','', word) for word in row.split() if word[0] == '#']))

def return_ats(row):
    return list(set(  [re.sub('[^\w\s]','', word) for word in row.split() if word[0] == '@']  ))

def remove_ats(row):
    return ' '.join([word for word in row.split() if word[0] != '@'])

def remove_stopwords(row):
    return ' '.join([word for word in row.split() if word not in stop])

def remove_https(row):
    return ' '.join([word for word in row.split() if word[0:6] != 'https:' and word[0:5] != 'http:'])


def lemmatized(row, lemmatizer):
    
    tokenized = word_tokenize(row)
    
    
    lemmatized_sentence = []
    for word, tag in pos_tag(tokenized):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word,pos))
    return lemmatized_sentence
    

In [165]:
# lower column names
df_tweets.columns = map(str.lower, df_tweets.columns)
df_stock.columns = map(str.lower, df_stock.columns)

df_tweets.rename(columns = {'text':'text_original'}, inplace = True)

df_tweets['text_modified'] = df_tweets['text_original'].str.lower() # małe znaki
df_tweets['hash'] = df_tweets['text_modified'].apply(return_hashes) # wybieranie unikalnych hashtagów (bez punktuacji)
df_tweets['at'] = df_tweets['text_modified'].apply(return_ats) # wybieranie unikalnych odnośników (bez punktuacji)
df_tweets['text_modified'] = df_tweets['text_modified'].apply(remove_https)
df_tweets['text_modified'] = df_tweets['text_modified'].str.replace('[^\w\s]','') # usuwanie punktuacji; można usuwać # ze zdań jeśli usunie się ten znak z regular expression
df_tweets['text_modified'] = df_tweets['text_modified'].apply(remove_stopwords)
df_tweets['text_lemmatized'] = df_tweets.apply(lambda x: lemmatized(x['text_modified'], WordNetLemmatizer()), axis=1)

df_tweets.created_at = pd.to_datetime(df_tweets.created_at)
df_tweets['date'] = df_tweets.created_at.dt.date
df_tweets['date'] = pd.to_datetime(df_tweets['date'])

df_stock.date = pd.to_datetime(df_stock.date)
df_stock['date'] = df_stock.date.dt.date


## Data preview

In [166]:
companies = df_tweets.company.unique()

for company in companies:
    dat1 = min(df_tweets.loc[df_tweets['company'] == company].created_at)
    dat2 = max(df_tweets.loc[df_tweets['company'] == company].created_at)
    
    print(f'Zakres datowy dla tweetów   {dat1}   -   {dat2}   :  {company}')

Zakres datowy dla tweetów   2020-02-03 11:30:17   -   2020-05-06 13:13:41   :  AstraZeneca
Zakres datowy dla tweetów   2020-02-03 16:39:56   -   2020-05-06 16:40:37   :  BayerPharma
Zakres datowy dla tweetów   2020-02-05 12:26:23   -   2020-05-07 15:41:11   :  GSK
Zakres datowy dla tweetów   2020-02-04 14:59:51   -   2020-05-07 18:10:43   :  Merck
Zakres datowy dla tweetów   2020-02-03 13:22:07   -   2020-05-07 19:55:40   :  Novartis
Zakres datowy dla tweetów   2020-02-17 13:55:00   -   2020-05-07 20:02:01   :  Pfizer
Zakres datowy dla tweetów   2020-02-03 09:01:21   -   2020-05-07 14:24:05   :  Roche
Zakres datowy dla tweetów   2020-02-03 10:03:21   -   2020-05-05 12:59:16   :  Sanofi


Ze względy na ograniczenie API Twittera dla każdej firmy pobranych zostało 200 tweetów. Zakres czasowy ich występowania różni się dla danych firm. Dlatego ustalony zostaje wspólny okres badania: od 1 lutego do 7 maja. Początego tego okresu można uznać za początek epidemii koronawirusa w Europie i Ameryce Północnej.

In [169]:
lower_date = datetime.strptime('2020-02-02','%Y-%m-%d').date()

df_tweets = df_tweets.loc[df_tweets['date'] >= lower_date]
df_stock = df_stock.loc[df_stock['date'] >= lower_date]

c:\python 3.7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.



Ponowne sprawdzenie zakresu dat, tym razem z ilością tweetów pozostałą po ograniczeniu zbioru danych dla każdej z firm.

In [149]:
companies = df_tweets.company.unique()

for company in companies:
    dat1 = min(df_tweets.loc[df_tweets['company'] == company].created_at)
    dat2 = max(df_tweets.loc[df_tweets['company'] == company].created_at)
    ilosc = len(df_tweets.loc[df_tweets['company'] == company])
    print(f'Zakres datowy dla tweetów   {dat1}   -   {dat2}   :  {company}; il. tweetów: {ilosc}')

Zakres datowy dla tweetów   2020-02-03 11:30:17   -   2020-05-06 13:13:41   :  AstraZeneca; il. tweetów: 107
Zakres datowy dla tweetów   2020-02-03 16:39:56   -   2020-05-06 16:40:37   :  BayerPharma; il. tweetów: 65
Zakres datowy dla tweetów   2020-02-05 12:26:23   -   2020-05-07 15:41:11   :  GSK; il. tweetów: 200
Zakres datowy dla tweetów   2020-02-04 14:59:51   -   2020-05-07 18:10:43   :  Merck; il. tweetów: 200
Zakres datowy dla tweetów   2020-02-03 13:22:07   -   2020-05-07 19:55:40   :  Novartis; il. tweetów: 106
Zakres datowy dla tweetów   2020-02-17 13:55:00   -   2020-05-07 20:02:01   :  Pfizer; il. tweetów: 200
Zakres datowy dla tweetów   2020-02-03 09:01:21   -   2020-05-07 14:24:05   :  Roche; il. tweetów: 195
Zakres datowy dla tweetów   2020-02-03 10:03:21   -   2020-05-05 12:59:16   :  Sanofi; il. tweetów: 193


Jak widać dla BayerPharmy ta ilość jest mniejsza niż poprzednio połowa. Ewentualnością będzie wykluczenie tej firmy z badania.

In [150]:
#https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

In [151]:
df_tweets.head(3)

,company,text_original,created_at,favourite_count,retweet_count,text_modified,hash,at,text_lemmatized,date
0,AstraZeneca,"Together with partners across industry, academia and government, we are taking a multipronged approach to helping patients around the world facing #COVID19. https://t.co/uQuHj6BkBN",2020-05-06 13:13:41,44,8,together partners across industry academia government taking multipronged approach helping patients around world facing covid19,[covid19],[],"[together, partner, across, industry, academia, government, take, multipronged, approach, help, patient, around, world, facing, covid19]",2020-05-06
1,AstraZeneca,"On #GivingTuesdayNow we stand with our partners @Plan_UK @Unicef_UK @ProjectHopeorg @NCDAlliance in their efforts responding to the unique health needs of groups vulnerable to #COVID19, such as those living with NCDs and young people. Get involved: https://t.co/YGRHLGqct6 https://t.co/vePEeAne49",2020-05-05 16:27:03,32,8,givingtuesdaynow stand partners plan_uk unicef_uk projecthopeorg ncdalliance efforts responding unique health needs groups vulnerable covid19 living ncds young people get involved,"[givingtuesdaynow, covid19]","[plan_uk, ncdalliance, unicef_uk, projecthopeorg]","[givingtuesdaynow, stand, partner, plan_uk, unicef_uk, projecthopeorg, ncdalliance, effort, respond, unique, health, need, group, vulnerable, covid19, living, ncds, young, people, get, involve]",2020-05-05
2,AstraZeneca,We’re #standingtogether4asthma with patients and the respiratory community during these times of uncertainty. Visit @WEF to learn more about what we’re doing to play our part in the fight against #COVID19: #WorldAsthmaDay \r\nhttps://t.co/fWE7ik8rNs https://t.co/Z54pyHBENq,2020-05-05 12:30:15,19,7,standingtogether4asthma patients respiratory community times uncertainty visit wef learn play part fight covid19 worldasthmaday,"[covid19, worldasthmaday, standingtogether4asthma]",[wef],"[standingtogether4asthma, patient, respiratory, community, time, uncertainty, visit, wef, learn, play, part, fight, covid19, worldasthmaday]",2020-05-05


In [152]:
df_stock.head(3)

,date,open,high,low,close,adj close,volume,company
63,2020-02-03,48.619999,48.930000,48.450001,48.509998,47.541225,1990900,AstraZeneca
64,2020-02-04,48.759998,49.090000,48.720001,48.759998,47.786232,1698800,AstraZeneca
65,2020-02-05,49.459999,49.849998,49.230000,49.730000,48.736862,2303000,AstraZeneca


## Sentyment

In [263]:
df_sentiment = pd.read_csv('input/sentiment.csv',encoding = "ISO-8859-1",header=None)
df_sentiment = df_sentiment[[0,5]]
df_sentiment.rename(columns={0:'sentiment',5:'text_original'},inplace=True)
df_sentiment['sentiment'] = df_sentiment['sentiment'].map({0:'negative',2:'neutral',4:'positive'})
# po 20 000 przykładków dla każdego z sentymentów
df_sentiment = pd.concat([df_sentiment.loc[df_sentiment['sentiment']=='negative'][0:20000],
                        df_sentiment.loc[df_sentiment['sentiment']=='positive'][0:20000] ])


In [264]:
%%time
df_sentiment['text_modified'] = df_sentiment['text_original'].str.lower()
df_sentiment['text_modified'] = df_sentiment['text_modified'].apply(remove_https)
df_sentiment['text_modified'] = df_sentiment['text_modified'].apply(remove_ats)
df_sentiment['text_modified'] = df_sentiment['text_modified'].str.replace('[^\w\s]','')
df_sentiment['text_modified'] = df_sentiment['text_modified'].apply(remove_stopwords)
df_sentiment['text_lemmatized'] = df_sentiment.apply(lambda x: lemmatized(x['text_modified'], WordNetLemmatizer()), axis=1)

Wall time: 1min 22s


In [285]:
dataset = []
for index, row in df_sentiment.iterrows():
    dataset.append(({token:True for token in row['text_lemmatized']},row['sentiment']) )
                        
random.shuffle(dataset)
train_set = dataset[:28000]
validation_set = dataset[28000:]
    

In [286]:
classifier = NaiveBayesClassifier.train(train_set)


In [287]:
print("Accuracy is:", classify.accuracy(classifier, validation_set))

Accuracy is: 0.7253333333333334


In [288]:
print(classifier.show_most_informative_features(10))

Most Informative Features
                  lonely = True           negati : positi =     20.8 : 1.0
                   tummy = True           negati : positi =     19.7 : 1.0
                    ache = True           negati : positi =     17.4 : 1.0
                     sad = True           negati : positi =     16.9 : 1.0
                  female = True           negati : positi =     16.1 : 1.0
                  bummer = True           negati : positi =     15.7 : 1.0
                     yum = True           positi : negati =     14.3 : 1.0
                    sick = True           negati : positi =     13.3 : 1.0
                     ugh = True           negati : positi =     12.9 : 1.0
                    argh = True           negati : positi =     12.6 : 1.0
None


## Wykresy danych z giełdy

In [153]:
fig = px.line(df_stock, x='date',y='close',color = 'company')
fig.show()

In [208]:
from collections import Counter, OrderedDict

In [178]:
date_intervals = pd.date_range(start=min(df_tweets.date), end=max(df_tweets.date), freq='3D')
df_tweets['date'] = pd.to_datetime(df_tweets['date'])


for i in range(0,len(date_intervals)-1):
    words = df_tweets.loc[(df_tweets['date'] >= date_intervals[i]) & (df_tweets['date'] < date_intervals[i+1])]['text_lemmatized'].sum()
    most_common= Counter(words).most_common()[:5]
    print(f'Dla okresu {date_intervals[i]} - {date_intervals[i+1]} najczęstsze słowa to:')
    print(most_common)
    print()

    

Dla okresu 2020-02-03 00:00:00 - 2020-02-06 00:00:00 najczęstsze słowa to:
[('worldcancerday', 102), ('thank', 69), ('join', 61), ('conversation', 58), ('cancer', 34)]

Dla okresu 2020-02-06 00:00:00 - 2020-02-09 00:00:00 najczęstsze słowa to:
[('result', 10), ('year', 10), ('today', 7), ('cancer', 7), ('treatment', 7)]

Dla okresu 2020-02-09 00:00:00 - 2020-02-12 00:00:00 najczęstsze słowa to:
[('woman', 9), ('science', 7), ('patient', 7), ('people', 6), ('disease', 6)]

Dla okresu 2020-02-12 00:00:00 - 2020-02-15 00:00:00 najczęstsze słowa to:
[('patient', 12), ('cancer', 12), ('disease', 7), ('make', 6), ('help', 6)]

Dla okresu 2020-02-15 00:00:00 - 2020-02-18 00:00:00 najczęstsze słowa to:
[('contact', 6), ('us', 5), ('local', 5), ('reach', 5), ('thanks', 4)]

Dla okresu 2020-02-18 00:00:00 - 2020-02-21 00:00:00 najczęstsze słowa to:
[('people', 7), ('learn', 7), ('cancer', 7), ('new', 6), ('health', 5)]

Dla okresu 2020-02-21 00:00:00 - 2020-02-24 00:00:00 najczęstsze słowa to:
[

Wygląda na to, że temat koronawirusa zaczął dominować w naszych Tweetach w okolicach 10 -15 marca. To w tych dniach m.in. W Polsce zarządzono lockdown. Jednak dnia 23 marca dla wszystkich wykresów giełdowych widać odbicie. Sprawdzę dla tej daty +/- 4 dni jakie były najczęstsze wyrażenia w tweetach.

In [188]:
date_intervals = pd.date_range(start='2020-03-19', end='2020-03-28', freq='D')
df_tweets['date'] = pd.to_datetime(df_tweets['date'])


for i in range(0,len(date_intervals)-1):
    words = df_tweets.loc[(df_tweets['date'] >= date_intervals[i]) & (df_tweets['date'] < date_intervals[i+1])]['text_lemmatized'].sum()
    try:
        most_common= Counter(words).most_common()[:5]
        print(f'Dla okresu {date_intervals[i]} - {date_intervals[i+1]} najczęstsze słowa to:')
        print(most_common)
        print()
    except:
        pass

Dla okresu 2020-03-19 00:00:00 - 2020-03-20 00:00:00 najczęstsze słowa to:
[('covid19', 6), ('us', 4), ('contact', 4), ('please', 4), ('vaccine', 4)]

Dla okresu 2020-03-20 00:00:00 - 2020-03-21 00:00:00 najczęstsze słowa to:
[('covid19', 8), ('contact', 6), ('info', 5), ('thanks', 4), ('coronavirus', 4)]

Dla okresu 2020-03-22 00:00:00 - 2020-03-23 00:00:00 najczęstsze słowa to:
[('covid19', 2), ('proud', 1), ('share', 1), ('donation', 1), ('500000', 1)]

Dla okresu 2020-03-23 00:00:00 - 2020-03-24 00:00:00 najczęstsze słowa to:
[('covid19', 9), ('patient', 7), ('help', 6), ('contact', 6), ('us', 6)]

Dla okresu 2020-03-24 00:00:00 - 2020-03-25 00:00:00 najczęstsze słowa to:
[('covid19', 8), ('disease', 4), ('test', 4), ('global', 3), ('patient', 3)]

Dla okresu 2020-03-25 00:00:00 - 2020-03-26 00:00:00 najczęstsze słowa to:
[('covid19', 10), ('help', 4), ('us', 4), ('contact', 4), ('million', 3)]

Dla okresu 2020-03-26 00:00:00 - 2020-03-27 00:00:00 najczęstsze słowa to:
[('covid19',

Dnia 26 marca pojawiają się nowe zwroty: treatment, trial, new, które to mogą sugerować prace nad leczeniem zakażenia. Warto pamiętać, że komunikaty na Twitterze mogą być opóźnione w stosunku do innych form komunikacji firm farmaceutycznych (szczególnie takich, jakie są obserwowane przez akcjonariuszy), w szczególności opóźnione do zakulisowych informacji pracowników firm.

Następnym krokiem będzie stworzenie wykresu zawierającego ilości występowań konkretnych (zadanych) słów w czasie.

In [260]:
def draw_words_occurence(words_list):
    
    date_intervals = pd.date_range(start=min(df_tweets.date), end=max(df_tweets.date), freq='1D')
    
    
    temp = pd.DataFrame(columns=['date','word','occurrence'])
    
    for i in range(0, len(date_intervals) -6):
        try:
            all_words = df_tweets.loc[(df_tweets['date'] >= date_intervals[i]) & (df_tweets['date'] < date_intervals[i+6])]['text_lemmatized'].sum()
            counted = Counter(all_words)
            for word in words_list:
                temp.loc[len(temp)]=[date_intervals[i],word,counted.get(word,0)/5]  
                
        except:
            pass
    print('\n\nWartości są pokazane na ruchomej średniej z 5 dni w celu wygładzenia linii wykresów')
    fig = px.line(temp, y = 'occurrence', x = 'date', color = 'word')
    fig.show()
    
        



In [261]:
draw_words_occurence(['covid19','treatment','trial','new','vaccine'])




Wartości są pokazane na ruchomej średniej z 5 dni


Teraz można zauważyć, że właśnie w okolicach 23 marca, kiedy większą reprezentację mają słowa treatment, tial, new, a także trochę vaccine, jest odbicie kursów akcji z dołka.

<br><br>Zamienię wykres zamknięcia w wykres zmian procentowych zamknięcia między kolejnymi dniami (powinno to w czytelny sposób pokazać dynamikę zmian akcji).

In [93]:
df = df_stock.copy()

df['date'] = pd.to_datetime(df['date'])
df['new_date'] = df['date'] + timedelta(days=1)
df = df.set_index('date').groupby('company').resample('D').ffill()
del df['company']

df.reset_index(inplace = True)
df['new_date'] = df['date'] + timedelta(days=1)

df = df.merge(df[['new_date','close','company']], left_on = ['date','company'], right_on =['new_date','company'], how = 'left')
df['close_diff'] = (df['close_y']/df['close_x'])*100


fig = px.line(df, x='date',y='close_diff',color = 'company')
fig.show()

Z wykresu powyżej można zauważyć, że największe wahania kursów były w miesiącu marcu. Może za to odpowiadać szok giełdowy jaki towrzyszył początkom epidemii.